<a href="https://colab.research.google.com/github/VenkateshDas/sqlalchemy/blob/master/sqlalchemy_sqlite_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
columns = ['name','grade','courses']
df = pd.DataFrame(columns = columns)

names = ['Mike','Alex','Liam','Emma','Noah','Olivia','William','Ava','James','Isabella','Oliver','Sophia','Benjamin','Charlotte','Elijah','Mia','Lucas','Amelia','Mason','Harper','Logan']
df['name'] = np.random.choice(list(names), 100)

grades = [1.0,2.0,3.0,4.0,5.0]
df['grade'] = np.random.choice(list(grades), 100)

courses = ['Programming', 'Physics', 'Science', 'Chemistry', 'Biology', 'English', 'Maths', 'German']
df['courses'] = np.random.choice(list(courses), 100)

In [5]:
df.head()

,name,grade,courses
0,Mike,4.0,English
1,Oliver,2.0,Science
2,Emma,1.0,Science
3,Mia,4.0,German
4,Mia,3.0,German


In [6]:
from sqlalchemy import create_engine
from sqlalchemy.sql import func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String , Float
import csv

engine = create_engine('sqlite:///:memory:')

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

class User(Base):
    __tablename__ = 'tasks'
    id = Column(Integer , primary_key = True)
    name = Column(String)
    grade = Column(Float)
    course = Column(String)

    def __repr__(self):
        return f'User {self.name}'

Base.metadata.create_all(engine) # initial table

for i in range(df.shape[0]):
    user = User(id = i , name=df['name'][i], grade = df['grade'][i] , course = df['courses'][i])
    session.add(user)

session.commit()

query = session.query(func.count(User.course),User.name).filter_by(grade = 1.0).group_by(User.name).having(func.count(User.course)>= 3.0)

print(query.all())

session.close()


[(3, 'Emma'), (3, 'Harper'), (3, 'Sophia')]


In [7]:
Base.metadata.drop_all(engine)

In [8]:
file = '/content/task.csv'
df_1 = pd.read_csv(file)

session = Session()

Base.metadata.create_all(engine)

for i in range(df_1.shape[0]):
    user = User(id = i , name=df_1['name'][i], grade = df_1['grade'][i] , course = df_1['course'][i])
    session.add(user)

session.commit()

query = session.query(func.count(User.course),User.name).filter_by(grade = 1.0).group_by(User.name).having(func.count(User.course)>= 3.0)

print(query.all())

session.close()




[(3, 'Milana')]


In [ ]:
Base.metadata.drop_all(engine)

In [ ]:
import time 

file = '/content/random_test_small.csv'
df_1 = pd.read_csv(file)

session = Session()

Base.metadata.create_all(engine)

start_time = time.time()

for i in range(df_1.shape[0]):
    user = User(id = i , name=df_1['name'][i], grade = df_1['grade'][i] , course = df_1['courses'][i])
    session.add(user)

session.commit()

end_time = time.time() - start_time

print("Total time to read and write the database : {} ".format(end_time))

query = session.query(func.count(User.course),User.name).filter_by(grade = 1.0).group_by(User.name).having(func.count(User.course)>= 3.0)

print(query.all())

session.close()

Total time to read and write the database : 157.12983560562134 
[(9676, 'Alex'), (9680, 'Amelia'), (9612, 'Ava'), (9636, 'Benjamin'), (9308, 'Charlotte'), (9412, 'Elijah'), (9541, 'Emma'), (9700, 'Harper'), (9665, 'Isabella'), (9497, 'James'), (9572, 'Liam'), (9489, 'Logan'), (9422, 'Lucas'), (9565, 'Mason'), (9716, 'Mia'), (9414, 'Mike'), (9332, 'Noah'), (9611, 'Oliver'), (9666, 'Olivia'), (9503, 'Sophia'), (9744, 'William')]


For a million data point with a file size of 20 MB it took 157 seconds to read and write the data. SInce it was done in Sqlite the in memory RAM usage also started shooting up. 

This is the primary reason to move to a postgreSQL database where the transactions can be done in the persistent memory efficiently. 

And for large datasets we can use the sqlalchemy ORM functionality of bulk inserts or bulk save. 